In [1]:
pip install openpyxl

     |████████████████████████████████| 243 kB 420 kB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=cba97617f83b1745f50e996b99ce28bebdd32f430a9add058e83fa7d9f37914d
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile
Note: you may need to restart the kernel to use updated packages.


## TÜRKİYE NÜFUS ANALİZİ(2020)
* Bu çalışmada 3 farklı veri seti kullanılmıştır.Bunların özellikleri;
* TUİK' in yayınlamış olduğu Türkiye'deki tüm illere ait toplam nüfus,erkek nüfus,kadın nüfus ve nüfus oranları bilgilerinin yer aldığı veri seti kullanılmıştır.
* Diğer veri setinde ise yine TUİK' in yayınlamış olduğu Türkiye'nin yıllara göre toplam erkek ve kadın nüfusları yer almkatadır.
* Son olarakta Türkiye'nin illerinin yer aldığı bir geojson veri seti kullanılmıştır.

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point
import json
import plotly.express as px
import branca.colormap as cmp
from branca.colormap import linear
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [3]:
df=pd.read_excel('../input/illere-gore-nufus-tr-2020/turkiye_nufus_2020.xlsx')

In [4]:
df.head()

,IL,TOPLAM NUFUS,ERKEK NUFUS,KADIN NUFUS
0,Adana,2258718,1127516,1131202
1,Adıyaman,632459,318755,313704
2,Afyon,736912,367399,369513
3,Ağrı,535435,276800,258635
4,Aksaray,423011,211232,211779


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
TOPLAM NUFUS,81.0,1.032276e+06,1.872576e+06,81910.0,284923.0,537762.0,1081065.0,15462452.0
ERKEK NUFUS,81.0,5.174813e+05,9.371936e+05,42002.0,148967.0,276776.0,552587.0,7750836.0
KADIN NUFUS,81.0,5.147948e+05,9.354143e+05,38628.0,139422.0,267536.0,528478.0,7711616.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IL            81 non-null     object
 1   TOPLAM NUFUS  81 non-null     int64 
 2   ERKEK NUFUS   81 non-null     int64 
 3   KADIN NUFUS   81 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.7+ KB


In [7]:
df['ERKEK ORAN']=df['ERKEK NUFUS']/df['TOPLAM NUFUS']
df['KADIN ORAN'] = df['KADIN NUFUS']/df['TOPLAM NUFUS']

In [8]:
geo=gpd.read_file('../input/turkey-city-geojson/tr-cities.json')
geo.head()

,name,number,geometry
0,Adana,1,"MULTIPOLYGON (((35.27903 36.64069, 35.28181 36..."
1,Adıyaman,2,"POLYGON ((38.65534 38.09172, 38.85888 38.09243..."
2,Afyon,3,"POLYGON ((31.36277 39.16468, 31.38636 39.11867..."
3,Ağrı,4,"POLYGON ((43.39566 39.92580, 43.39423 39.79461..."
4,Aksaray,68,"POLYGON ((34.20521 38.84115, 34.26257 38.79367..."


In [9]:
geo= geo.rename(columns = {"name":"IL"})
final_df =geo.merge(df, on = "IL")
final_df.head()

,IL,number,geometry,TOPLAM NUFUS,ERKEK NUFUS,KADIN NUFUS,ERKEK ORAN,KADIN ORAN
0,Adana,1,"MULTIPOLYGON (((35.27903 36.64069, 35.28181 36...",2258718,1127516,1131202,0.499184,0.500816
1,Adıyaman,2,"POLYGON ((38.65534 38.09172, 38.85888 38.09243...",632459,318755,313704,0.503993,0.496007
2,Afyon,3,"POLYGON ((31.36277 39.16468, 31.38636 39.11867...",736912,367399,369513,0.498566,0.501434
3,Ağrı,4,"POLYGON ((43.39566 39.92580, 43.39423 39.79461...",535435,276800,258635,0.516963,0.483037
4,Aksaray,68,"POLYGON ((34.20521 38.84115, 34.26257 38.79367...",423011,211232,211779,0.499353,0.500647


In [10]:
final_df = GeoDataFrame(final_df, crs="EPSG:4326")

In [11]:
m = folium.Map(location=[37.935,35.173],zoom_start=5)

## TOPLAM NÜFUSA GÖRE CHOROPLETH HARİTA

In [19]:
bins = list(df["TOPLAM NUFUS"].quantile([0, 0.25, 0.5, 0.75, 1]))
choropleth=folium.Choropleth(
    geo_data=final_df,
    name='choropleth',
    data=final_df,
    columns=['IL','TOPLAM NUFUS'],
    key_on='feature.properties.IL',
    fill_color='OrRd',
    bins=bins,
    nan_fill_color='black',
    fill_opacity=0.7,
    line_opacity=0.5,)
for key in choropleth._children:
    if key.startswith('color_map'):
        del(choropleth._children[key])
choropleth.add_to(m)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['IL','TOPLAM NUFUS'],labels=False))
colormap = cmp.LinearColormap(colors=['white','yellow','#fbff00','#ff8800','#ff3700','orange','red','#ff0400'], index=[0,284923,537762,1081065,1546245],vmin=0,vmax=1546245)
colormap.caption = 'TOPLAM NÜFUS (2020)'
colormap.add_to(m)
m

## ERKEK NÜFÜS ORANLARINA GÖRE CHOROPLETH HARİTA

In [12]:
bins = list(df["ERKEK ORAN"].quantile([0, 0.25, 0.5, 0.75, 1]))
choropleth=folium.Choropleth(
    geo_data=final_df,
    name='choropleth',
    data=final_df,
    legend_name='ERKEK NÜFUS ORANI',
    columns=['IL','ERKEK ORAN'],
    key_on='feature.properties.IL',
    fill_color='YlGn',
    bins=bins,
    nan_fill_color='black',
    fill_opacity=0.7,
    line_opacity=0.5,)
choropleth.add_to(m)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['IL','ERKEK NUFUS'],labels=False))
m

## KADIN NÜFÜS ORANLARINA GÖRE CHOROPLETH HARİTA

In [12]:
bins = list(df["KADIN ORAN"].quantile([0, 0.25, 0.5, 0.75, 1]))
choropleth=folium.Choropleth(
    geo_data=final_df,
    name='choropleth',
    data=final_df,
    columns=['IL','KADIN ORAN'],
    key_on='feature.properties.IL',
    fill_color='PuRd',
    bins=bins,
    legen_name='KADIN NÜFUS ORANI',
    nan_fill_color='black',
    fill_opacity=0.7,
    line_opacity=0.5,)
choropleth.add_to(m)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['IL','KADIN NUFUS'],labels=False))
m

In [13]:
import plotly.express as px
import plotly
import matplotlib.pyplot as plt

In [14]:
top_otuz=df[['TOPLAM NUFUS','IL']].sort_values('TOPLAM NUFUS',ascending=False).head(30)

In [19]:
fig=px.bar(top_otuz,x='TOPLAM NUFUS',y='IL',color='TOPLAM NUFUS',height=700,
           title='NÜFUSU EN ÇOK OLAN İLK OTUZ İL',color_continuous_scale=plotly.express.colors.sequential.Blugrn)
fig.show()

In [20]:
son_otuz=df[['TOPLAM NUFUS','IL']].sort_values('TOPLAM NUFUS',ascending=True).head(30)

In [44]:
fig=px.bar(son_otuz,x='TOPLAM NUFUS',y='IL',color='TOPLAM NUFUS',height=700,
           title='NÜFUSU AZ OLAN SON OTUZ İL',
           color_continuous_scale=plotly.express.colors.sequential.OrRd)
fig.show()

In [22]:
a = pd.Series(df['TOPLAM NUFUS'].sum(), index = ['TOPLAM NUFUS'])
b = pd.Series(df['ERKEK NUFUS'].sum(), index = ['ERKEK NUFUS'])
c = pd.Series(df['KADIN NUFUS'].sum(), index = ['KADIM NUFUS'])
print(a)
print(b)
print(c)

TOPLAM NUFUS    83614362
dtype: int64
ERKEK NUFUS    41915985
dtype: int64
KADIM NUFUS    41698377
dtype: int64


In [23]:
tdf = pd.DataFrame({'CINSIYET':['ERKEK', 'KADIN'], 'TOPLAM NUFUS':[41915985,41698377]})

In [24]:
fig = px.pie(tdf, values='TOPLAM NUFUS', names='CINSIYET', title='CİNSİYETE GÖRE TOPLAM NÜFUS')
fig.show()

In [16]:
ydf=pd.read_excel("../input/yillara-gore-nufus-tr/yllara_gre_nfus_tr.xlsx")

In [16]:
ydf.head()

,YIL,TOPLAM NUFUS,ERKEK NUFUS,KADIN NUFUS
0,2020,83614362,41915985,41698377
1,2019,83154997,41721136,41433861
2,2018,82003882,41139980,40863902
3,2017,80810525,40535135,40275390
4,2016,79814871,40043650,39771221


In [18]:
ydf.describe().T

,count,mean,std,min,25%,50%,75%,max
YIL,14.0,2.013500e+03,4.183300e+00,2007.0,2010.25,2013.5,2016.75,2020.0
TOPLAM NUFUS,14.0,7.723163e+07,4.302889e+06,70586256.0,73973308.25,77181884.0,80561611.50,83614362.0
ERKEK NUFUS,14.0,3.875694e+07,2.146323e+06,35376533.0,37165625.00,38728831.0,40412263.75,41915985.0
KADIN NUFUS,14.0,3.847468e+07,2.156866e+06,35209723.0,36807683.25,38453053.0,40149347.75,41698377.0


In [19]:
ydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   YIL           14 non-null     int64
 1   TOPLAM NUFUS  14 non-null     int64
 2   ERKEK NUFUS   14 non-null     int64
 3   KADIN NUFUS   14 non-null     int64
dtypes: int64(4)
memory usage: 576.0 bytes


In [17]:
fig = px.line(ydf, x='YIL', y=df.columns[2:4],
              title='YILLARA GÖRE ERKEK ve KADIN TOPLAM NÜFUS DEĞERLERİ')
fig.show()